# Querying Database

### You can call .query method to actually see the query that get executed behind the schene

In [ ]:
Post.objects.all().order_by('first name').query

# SELECT first_name, last_name FROM POST ORDER BY first_name;

### Model manager

- Interface through which query operation can be performed
- at least one manager exist for ever model
- manager with name ```objects``` to every Django model class
- We can also create our own model manager or modify existing model manager.

```
    > python manage.py shell

    > from models impport User

    > User.objects 
    #should return Manager Object

```

## A. Creating Object
---
1. save
2. create
3. get_or_create
4. add()
4. update_or_create
5. bulk_create
6. bulk_update
---
### <u> 1. save(force_insert=False,force_update=False, update_fields = None) </u>
- A model class represents a database table, and an instance of that class represents a particular record in db.

- save() method doesnot return any value

params:
 >   - force_insert->Bool = Force INSERT query and not fall back to UPDATE
 >   - force_update->Bool = Update if possible, but not insert new ROW.
 >   - update_field = Spicfy which fields to save, automatic force_update = True

eg: 
    
```
   product.name = 'ice cream'
   product.save(update_fields = ['name'])
```


In [ ]:
# eg
a = User.objects.get(username='django') # get a user instance 
b = Blog(name='Interesting title', author = a) # perform INSERT operation

b.id #returns none because b doesnot have an id yet
b.save() # Django doesnot hit the database until you explictly call save()

b.id # returns id of your new object



### 2. <u>create(through_defaults=None, **kwargs)¶</u>

- A conventional method of creating object and saving it all in one step.
-  Returns the newly created object:

```p = Blog.objects.create(name='Interesting title', author = a)```
* This step is same as above example

<b> Note : in save() method if you specify existing id(primary key) then it will update the ROW, 
 how ever in create(), if you explictly specify existing pk, it will return IntegrityError since pk must be unique.

### 3. <u>get_or_create(default=None,**kwargs)</u>
- Looks an object with the given kwargs , creating one if not found.
- return tuple of (object, created) , where object is created object and created is bool specifying whether a new object
    was created.
- This is meant to prevent duplicate objects from being created when requests are made in parallel, and as a shortcut to boilerplatish code. For example:

In [ ]:
try:
    obj = Person.objects.get(first_name='John', last_name='Lennon')
except Person.DoesNotExist:
    obj = Person(first_name='John', last_name='Lennon', birthday=date(1940, 10, 9))
    obj.save()

Here, with concurrent requests, multiple attempts to save a Person with the same parameters may be made. To avoid this race condition, the above example can be rewritten using get_or_create() like so:

In [ ]:
obj, created = Person.objects.get_or_create(
    first_name='John',
    last_name='Lennon',
    defaults={'birthday': date(1940, 10, 9)},
)


## 4. add(*objs, bulk=True, through_defaults=None)

Adds the specified model objects to the related object set.

```
>>> b = Blog.objects.get(id=1)
>>> e = Entry.objects.get(id=234)
>>> b.entry_set.add(e) # Associates Entry e with Blog b.
```

In the example above, in the case of a ForeignKey relationship, QuerySet.update() is used to perform the update. This requires the objects to already be saved.

You can use the bulk=False argument to instead have the related manager perform the update by calling e.save().

Using add() with a many-to-many relationship, however, will not call any save() methods (the bulk argument doesn’t exist), but rather create the relationships using QuerySet.bulk_create(). If you need to execute some custom logic when a relationship is created, listen to the m2m_changed signal, which will trigger pre_add and post_add actions.

Using add() on a relation that already exists won’t duplicate the relation, but it will still trigger signals.

For many-to-many relationships add() accepts either model instances or field values, normally primary keys, as the *objs argument.

Use the through_defaults argument to specify values for the new intermediate model instance(s), if needed. You can use callables as values in the through_defaults dictionary and they will be evaluated once before creating any intermediate instance(s).

## <u> B. Retriving Objects</u>

In [ ]:
#all() -> for all object
User.objects.all() # retrieve all users objects from  db
User.objects.all().query # print the query used to retrieve all users

# get() -> for only one object
try:

    User.objects.get(username='django') # return only one object
except Model.DoesNotExist:
    pass
except Model.MultipleObjectReturned: 
    pass

#filter() for muliple objects
User.objects.filter(role='dev')

### order_by,values, only, difer

In [ ]:
Post.objects.all().order_by('first_name') # in ascending
Post.objects.all().order_by('-first_name') # in desceding


#values -> return dictionary instead of tuple of object
Post.objects.values('id','title')
# [{'id':5,'title':'AAAAA'} , ....]

# only -> extract only provided column from database
Post.objects.only('title','id','desc')

#defer -> exclude the columns specified(inverse of only)
Post.objects.differ('id','title')





## <u>C. Update data in db</u>

In [ ]:
#method 1
# It is used for update but is inefficient because it create object in memory. instead use .create() method
user  = User.objects.get(pk=1)
user.first_name = 'Django'
user.save()

#method 2
# efficient way/ also prevent race condition
Entry.objects.filter(id=1).update(first_name='django')

OR

 To turn comments off for all blog entries published in 2010, you could do this

In [ ]:
Entry.objects.filter(pub_date__year=2010).update(comments_on=False)


(This assumes your Entry model has fields pub_date and comments_on.)

The update() method is applied instantly, and the only restriction on the QuerySet that is updated is that it can only update columns in the model’s main table, not on related models. You can’t do this, for example:

In [ ]:
Entry.objects.update(blog__name='foo') # Won't work!

Finally, realize that update() does an update at the SQL level and, thus, does not call any save() methods on your models, nor does it emit the pre_save or post_save signals (which are a consequence of calling Model.save()). If you want to update a bunch of records for a model that has a custom save() method, loop over them and call save(), like this:

In [ ]:
for e in Entry.objects.filter(pub_date__year=2010):
    e.comments_on = False
    e.save()

##  <u> E. Delete data from db</u>


In [ ]:
user  = User.objects.get(pk=1)
user.delete()

# or

User.objects.filter(id=1).delete()


The delete() method does a bulk delete and does not call any delete() methods on your models. It does, however, emit the pre_delete and post_delete signals for all deleted objects (including cascaded deletions).



---

# Migrations

1. makemigrations - create new migrations based on model
2. migrate - apply and un apply migrations
3. showmigrations - list a projects migration and their status
4. sqlmigrate  - display sql statements for a migration


# Model field types

[Descriptions of types](https://www.geeksforgeeks.org/django-model-data-types-and-fields-list/)

1. CharField
2. TextField
3. DateField
4. TimeField
5. DateTimeField
6. SlugField 
7. ImageField
8. EmailField
9. FileField
10. JSONField
11. URLField
12. ForeignKey
13. ManyToManyField
 and others ...

## Primary Key in django
By default django gives each model an auto_incrementing primary key with the type specified per app in AppConfig.default_auto_field or globally in DEFAULT_AUTO_FIELD setting.py

For example ```id=models.BigAutoField(primary_key=True)```

# Model Field Options

null -> Bool
blank -> Bool
choices -> set of choices
default -> 
editable -> Bool (user wont be able to edit this field)
error_message -> 
help_text -> (usually format of this field)
primary_key
unique -> Bool
verbose_name -> Optional first positional argument, available to all except Many2many,1to1,many2one,
>    ``` name = models.CharField('full name',max_le....)```<br>
>    ``` full_name = models.CharField(max_length = ....)```<br>
>            -> verbose name  = full_name -> full name


In [ ]:
# eg:

from django.db import models
class User(models.Model):


    ROLE_CHOICES = (
        # Left is stored in DB, Right is displayed in frontend
        ('ADMIN','ADMIN')
        ('DEV','DEVELOPER')
        ('SU','SUPERADMIN')
        ('MGMT','MANAGEMENT')
    )

    first_name = models.CharField(max_length=250,
        error_messages={
            'max_length':'You cant add more than 250 char'
        }
    )
    body = models.TextField()
    joined_on = models.DateTimeField(auto_now_add=True)
    updated_on = models.DateTimeField(auto_now=True)
    slug = models.SlugField(max_length=300,unique=True,null=True,blank=True)
    is_active = models.BooleanField(default=True)
    role = models.CharField(max_length=5,choices=ROLE_CHOICES, default='DEV')



# Relationship in Django

# 1.Many-to-many fields:
This is used when one record of a model A is related to multiple records of another model B and vice versa. For example – a model Book has many-to-many relationship with a model Author, i.e. an book can be written by multiple authors and an author can write multiple books. Many-to-many relations are defined using ManyToManyField field of django.db.models

In [ ]:
from django.db import models
  
class Author(models.Model):
    name = models.CharField(max_length = 100)
    desc = models.TextField(max_length = 300)
  
class Book(models.Model):
    title = models.CharField(max_length = 100)
    desc = models.TextField(max_length = 300)
    authors = models.ManyToManyField(Author)

It is a good practice to name the many-to-many field with the plural version of the related model, lowercase. It doesn’t matter which of the two models contain the many-to-many field, but it shouldn’t be put in both the models.

### NOTE:

- By default, django creates an intermediate class for many to many relationship. This is the only 
  way database knows many to many relationship.
-  We need to specify the intermediate model via through parameter in ManyToManyField.

In [ ]:
class Item(models.Model):
    name = models.CharField(max_length = 128)
    price = models.DecimalField(max_digits = 5, decimal_places = 2)
 
    def __str__(self):
        return self.name
 
class Customer(models.Model):
    name = models.CharField(max_length = 128)
    age = models.IntegerField()
    items_purchased = models.ManyToManyField(Item, through = 'Purchase')
 
    def __str__(self):
        return self.name
 
class Purchase(models.Model):
    item = models.ForeignKey(Item, on_delete = models.CASCADE)
    customer = models.ForeignKey(Customer, on_delete = models.CASCADE)
    date_purchased = models.DateField()
    quantity_purchased = models.IntegerField()
    # We can make customer and item unique by defining following meta class
    class Meta:
        unique_together = ('item','customer',)


# 2. Many-to-One

This is used when one record of a model A is related to multiple records of another model B. For example – a model Song has many-to-one relationship with a model Album, i.e. an album can have many songs, but one song cannot be part of multiple albums. Many-to-one relations are defined using ForeignKey field of django.db.models.



In [ ]:
class Album(models.Model):
    title = models.CharField(max_length = 100)
    artist = models.CharField(max_length = 100)
  
class Song(models.Model):
    title = models.CharField(max_length = 100)
    album = models.ForeignKey(Album, on_delete = models.CASCADE)

It is a good practice to name the many-to-one field with the same name as the related model, lowercase.

# 3. One-to-one fields:
This is used when one record of a model A is related to exactly one record of another model B. This field can be useful as a primary key of an object if that object extends another object in some way. For example – a model Car has one-to-one relationship with a model Vehicle, i.e. a car is a vehicle. One-to-one relations are defined using OneToOneField field of django.db.models.

Below is an example to demonstrate the same.

In [2]:
class Vehicle(models.Model):
    reg_no = models.IntegerField()
    owner = models.CharField(max_length = 100)
  
class Car(models.Model):
    vehicle = models.OneToOneField(Vehicle, 
          on_delete = models.CASCADE, primary_key = True)
    car_model = models.CharField(max_length = 100)

NameError: name 'models' is not defined

# Data integrity options:
Since we are creating models which depend on other models, we need to define the behavior of a record in one model when the corresponding record in the other is deleted. This is achieved by adding an optional on_delete parameter in the relational field, which can take the following values:

1. <u>on_delete</u> = models.CASCADE – This is the default value. It automatically deletes all the related records when a record is deleted.(e.g. when an Album record is deleted all the Song records related to it will be deleted)
2. <u>on_delete</u> = models.PROTECT – It blocks the deletion of a record having relation with other records.(e.g. any attempt to delete an Album record will be blocked)
3. <u>on_delete</u>  = models.SET_NULL – It assigns NULL to the relational field when a record is deleted, provided null = True is set.
4. <u>on_delete</u>  = models.SET_DEFAULT – It assigns default values to the relational field when a record is deleted, a default value has to be provided.
5. <u>on_delete</u>  = models.SET() – It can either take a default value as parameter, or a callable, the return value of which will be assigned to the field.
6. <u>on_delete</u>  = models.DO_NOTHING – Takes no action. Its a bad practice to use this value.


# Model Methods




In [ ]:
from django.utils.text import slugify
class Company(models):
    name = models.CharField(max_length=100)
    # A slug is a short label for something, containing only letters, numbers, underscores or hyphens
    slug = models.CharField(max_length=100)

    def __str__(self):
        '''
        Str reprensentation used by admin panel
        '''
        return self.name
    
    def save(self,*args,**kwargs):
        '''
        Override method, run when save is called. eg. User.save()
        '''
        if not self.slug:

            self.slug = slugify(self.name) + 'a_unique_value_placeholder'
        return super().save(*args,**kwargs)

    class meta:
        '''

            https://www.geeksforgeeks.org/meta-class-in-models-django/#:~:text=Model%20Meta%20is%20basically%20the,Meta%20class%20in%20your%20model.


            metadata is anything thats not a field.such as ordering options, db tablename, human readabale signature
            pluarl_names( verbose_name and verbose_name_plural)


            OPTIONAL
        '''

        abstract = True
        verbose_name = 'Comparny'
        db_table = 'Company'
        order_with_respect_to = ''
        ordering = ''
        permissions = ''
        indexes = ''
        unique_together = ''
        index_together = ''
        ..